In [1]:
import numpy as np
import pandas as pd
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV
)
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.theta import ThetaForecaster

from sktime.forecasting.arima import ARIMA
# from sktime.forecasting.statsforecast import StatsForecastARIMA
from sktime.forecasting.compose import MultiplexForecaster
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

np.random.seed(42)
y = pd.Series(np.sin(np.arange(100) * 0.1) + np.random.normal(0, 0.1, 100))

# -------------------------
# 2. Define forecasting horizon
# -------------------------
fh = np.arange(1, 5)   # forecast 4 steps ahead

forecasters = MultiplexForecaster(
    forecasters=[
        ("naive", NaiveForecaster(strategy="last")),
        ("arima", ARIMA(order=(1,1,0))),
        ("ets", ExponentialSmoothing(trend="add", seasonal="add", sp=12)),
        ("theta", ThetaForecaster(sp=12))
    ],
    selected_forecaster="naive"   # default, will be tuned later
)


ImportError: DLL load failed while importing lib: The specified module could not be found.

In [ ]:
param_grid = [
    {"selected_forecaster": ["naive"],
     "naive__strategy": ["last", "mean"]},
    
    {"selected_forecaster": ["arima"],
     "arima__order": [(1,0,0), (2,1,0), (1,1,1)]}
]


In [ ]:
from sktime.forecasting.model_selection import ExpandingWindowSplitter, ForecastingGridSearchCV

cv = ExpandingWindowSplitter(initial_window=50, fh=[1,2,3])  # train expands, test moves forward

gscv = ForecastingGridSearchCV(
    forecaster=forecasters,
    param_grid=param_grid,
    cv=cv,
    scoring="mean_absolute_percentage_error",   # metric to decide best
    n_jobs=-1
)

gscv.fit(y)   # y = your time series


In [ ]:
print("Best Parameters:", gscv.best_params_)
print("Best Score:", gscv.best_score_)
print("Best Forecaster:", gscv.best_forecaster_)


In [ ]:
import numpy as np
import pandas as pd
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV
)
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.compose import MultiplexForecaster
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

# -------------------------
# 1. Create synthetic data
# -------------------------
np.random.seed(42)
y = pd.Series(np.sin(np.arange(100) * 0.1) + np.random.normal(0, 0.1, 100))

# -------------------------
# 2. Define forecasting horizon
# -------------------------
fh = np.arange(1, 5)   # forecast 4 steps ahead

# -------------------------
# 3. Define models
# -------------------------
forecasters = MultiplexForecaster(
    forecasters=[
        ("naive", NaiveForecaster(strategy="last")),
        ("arima", ARIMA(order=(1, 0, 0)))
    ],
    selected_forecaster="naive"
)

# -------------------------
# 4. Define parameter grid
# -------------------------
param_grid = [
    {
        "selected_forecaster": ["naive"],
        "naive__strategy": ["last", "mean"]
    },
    {
        "selected_forecaster": ["arima"],
        "arima__order": [(1,0,0), (2,1,0), (1,1,1)]
    }
]

# -------------------------
# 5. Define CV splitter
# -------------------------
cv = ExpandingWindowSplitter(initial_window=70, step_length=5, fh=fh)

# -------------------------
# 6. Grid Search
# -------------------------
gscv = ForecastingGridSearchCV(
    forecaster=forecasters,
    param_grid=param_grid,
    cv=cv,
    scoring="mean_absolute_percentage_error",
    n_jobs=-1
)

gscv.fit(y)

print("Best Params:", gscv.best_params_)
print("Best Forecaster:", gscv.best_forecaster_)
print("Best Score:", gscv.best_score_)

# -------------------------
# 7. Forecast with best model
# -------------------------
y_pred = gscv.predict(fh)
print("\nForecast:", y_pred)
